In [1]:
year_testing=5

In [2]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import DateOffset
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
comex_df=pd.read_csv('/home/andre301267/git/Pricing-Fertilizer/DB/temp/imp_fert_ncm_quanti.csv')

Filtragem preliminar

In [3]:
comex_df['date']=pd.to_datetime(comex_df.year.astype(str)+'-'+comex_df.month.astype(str)+'-01')

In [4]:
month=comex_df[comex_df.date==comex_df.date.max()].month.max()
month

np.int64(8)

In [5]:
# Filtra produto e UF
comex_df=comex_df[(comex_df.Product.isin(['MOP']))&(comex_df.UF.isin(['BA']))&(comex_df.month.isin([month]))]

In [6]:
# filtra km
filtra_km=comex_df[['Product','km']].drop_duplicates().sort_values(by=['Product','km'],ascending=False).drop_duplicates(subset='Product')
comex_df=comex_df.merge(filtra_km)

Matriz Esparsa

In [7]:
date_min=comex_df.date.min()
date_max=comex_df.date.max()

In [8]:
years=pd.DataFrame({'year':np.arange(comex_df.year.min(),comex_df.year.max()+1)}).assign(key=0)
months=pd.DataFrame({'month':np.arange(1,13)}).assign(key=0)
calendar=years.merge(months,on='key')
full=comex_df[['Product','UF']].drop_duplicates().assign(key=0).merge(calendar,on='key').drop('key',axis=1)
comex_df=comex_df.merge(full)

In [9]:
comex_df[['month','Product','UF']]=comex_df[['month','Product','UF']].astype('category')
comex_df=comex_df.groupby(by=['year','month','Product','UF'],observed=False).kg.sum().reset_index()

In [10]:
comex_df['date']=pd.to_datetime(comex_df.year.astype(str)+'-'+comex_df.month.astype(str)+'-01')

In [11]:
comex_df=comex_df[(comex_df.date>=date_min)&(comex_df.date<=date_max)]

Dummies

In [12]:
comex_df=pd.get_dummies(comex_df,prefix_sep='~')

# Classify

In [13]:
comex_df=comex_df.assign(oc=0)
comex_df.loc[comex_df.kg>0,'oc']=1

In [14]:
comex_df_t=comex_df.copy()
for r in range(1,year_testing+1):
    train=comex_df_t[comex_df_t.date<comex_df_t.date.max()]
    test=comex_df_t[comex_df_t.date==comex_df_t.date.max()]
    X_train=train.drop(['kg','oc','date'],axis=1)
    y_train=train.oc
    X_test=test.drop(['kg','oc','date'],axis=1)
    y_test=test.oc
    from sklearn.ensemble import RandomForestClassifier as rfc
    model=rfc()
    model.fit(X_train,y_train)
    y_pred=model.predict(X_test)
    test['y_pred']=y_pred
    if r==1:
        results=test
    else:
        results=pd.concat([results,test])
    
    comex_df_t=comex_df_t[comex_df_t.date<=comex_df_t.date.max()-DateOffset(years=1)]

# Regression

In [15]:
comex_df=results.merge(comex_df,how='outer')

In [16]:
comex_df=comex_df[(comex_df.oc==1)&(comex_df.y_pred.isin([1,np.nan]))]

In [17]:
comex_df_t=comex_df.copy()
features_df=pd.DataFrame()
for r in range(1,year_testing+1):
    train=comex_df_t[comex_df_t.date<comex_df_t.date.max()]
    test=comex_df_t[comex_df_t.date==comex_df_t.date.max()]
    X_train=train.drop(['kg','oc','date'],axis=1)
    y_train=train.kg
    X_test=test.drop(['kg','oc','date'],axis=1)
    y_test=test.kg
    from sklearn.ensemble import RandomForestRegressor as rfr
    model=rfr()
    model.fit(X_train,y_train)
    t=pd.DataFrame({'names':model.feature_names_in_,'importances':model.feature_importances_})
    features_df=pd.concat([features_df,t])
    y_pred=model.predict(X_test)
    test['y_pred']=y_pred
    if r==1:
        results=test
    else:
        results=pd.concat([results,test])
    
    comex_df_t=comex_df_t[comex_df_t.date<=comex_df_t.date.max()-DateOffset(years=1)]


display(features_df.groupby('names').importances.mean().reset_index().sort_values(by='importances',ascending=False))

,names,importances
4,year,1.0
0,Product~MOP,0.0
1,UF~BA,0.0
2,month~8,0.0
3,y_pred,0.0


In [18]:
o=results[results.columns[~results.columns.str.contains('~')]]

In [19]:
prdt=pd.from_dummies(results[results.columns[results.columns.str.contains('Pro')]],sep='~')

In [20]:
month=pd.from_dummies(results[results.columns[results.columns.str.contains('month')]],sep='~')

In [21]:
uf=pd.from_dummies(results[results.columns[results.columns.str.contains('UF')]],sep='~')

In [22]:
results=pd.concat([o,prdt,month,uf],axis=1)

In [23]:
mes=results.month.unique()[0]

In [24]:
results[(results.Product=='MOP')&(results.month==mes)&(results.UF=='BA')]

,year,kg,date,oc,y_pred,Product,month,UF
24,2024,73074320.0,2024-08-01,1,37491896.28,MOP,8,BA
23,2023,37734795.0,2023-08-01,1,39011307.08,MOP,8,BA
22,2022,46383060.0,2022-08-01,1,28677440.24,MOP,8,BA
21,2021,27695000.0,2021-08-01,1,29480712.29,MOP,8,BA
20,2020,15415000.0,2020-08-01,1,59775644.73,MOP,8,BA


In [25]:
from sklearn.metrics import root_mean_squared_error as rse
results['rse']=rse(results.kg,results.y_pred)

In [26]:
int(results.rse.unique()/10**6)

26